Parse the profiler output to compare output rows and op times for GPU shuffled hash join operations.
On the CPU the equivalent is a sort merge join.

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

profile_dirs = ["/opt/data/profiles/dataproc-gpu-sf100",
                "/opt/data/profiles/dataproc-gpu-sf3k",
                "/opt/data/profiles/dataproc-gpu-sf5k",
                "/opt/data/profiles/dataproc-gpu-sf10k"]
scale_factors = ["100", "3K", "5K", "10K"]

In [ ]:
num_scale_factors = len(profile_dirs)
output_rows = [[] for _ in range(num_scale_factors)]
op_times = [[] for _ in range(num_scale_factors)]
for i, profile_dir in enumerate(profile_dirs):
    apps = os.listdir(profile_dir)
    apps.sort()
    for app in apps:
        print(f"Processing gpu profile {profile_dir}/{app}")
        sql_info = pd.read_csv(f"{profile_dir}/{app}/sql_plan_metrics_for_application.csv")
        condition = (sql_info['nodeName'] == 'GpuShuffledHashJoin')
        filtered = sql_info[condition]
        grouped = filtered.groupby("nodeID")
        for _, group in grouped:
            output_rows_for_op = 0
            op_time_for_op = 0
            for row in group.itertuples():
                if row.name == "output rows":
                    output_rows_for_op = row.total
                elif row.name == "op time":
                    op_time_for_op = row.total / 1000000.
            if output_rows_for_op != 0 and op_time_for_op != 0:
                output_rows[i].append(output_rows_for_op)
                op_times[i].append(op_time_for_op)
print([len(x) for x in output_rows])
print([len(x) for x in op_times])

In [ ]:
for i in range(num_scale_factors):
    # Create a scatter plot
    plt.scatter(output_rows[i], op_times[i])
    plt.xscale('log')
    plt.yscale('log')
    
    # Adding title and labels (optional)
    plt.title(f"GPU Shuffled Hash Join Op Times - SF{scale_factors[i]}")
    plt.xlabel("Output Rows")
    plt.ylabel("GPU Op Time (ms)")
    
    # Show the plot
    plt.show()

In [ ]:
for i in range(num_scale_factors):
    plt.scatter(output_rows[i], op_times[i], label=f"SF{scale_factors[i]}")

plt.xscale('log')
plt.yscale('log')
plt.title("GPU Shuffled Hash Join Op Times - Combined")
plt.xlabel("Output Rows")
plt.ylabel("GPU Op Time (ms)")
plt.legend()

# Show the plot
plt.show()